In [13]:
import collections
import math
import numpy as np
import pickle
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)

Using cuda


In [24]:
folder_path = "C:/Users/Kieran/Documents/Master Thesis Data/Datasets/ChangeCoords"

dataframes = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)
        
data = pd.concat(dataframes, ignore_index=True)
data = data.drop(columns = ['frame_number'])
data

,"('nose', 'x')","('nose', 'y')","('nose', 'likelihood')","('H1R', 'x')","('H1R', 'y')","('H1R', 'likelihood')","('H2R', 'x')","('H2R', 'y')","('H2R', 'likelihood')","('H1L', 'x')",...,"('tail', 'x')","('tail', 'y')","('tail', 'likelihood')","('S2', 'x')","('S2', 'y')","('S2', 'likelihood')","('S1', 'x')","('S1', 'y')","('S1', 'likelihood')",mouse_no
0,0.000000,0.000000,0.999969,0.000000,0.000000,0.999760,0.000000,0.000000,0.996968,0.000000,...,0.000000,0.000000,0.998689,0.000000,0.000000,0.994912,0.000000,0.000000,0.995648,11.4
1,0.444275,2.021118,0.999950,0.536194,1.981354,0.999568,4.448181,4.355835,0.997916,0.535675,...,1.670120,0.212906,0.999059,0.595001,0.076477,0.996069,3.630341,2.596069,0.993003,11.4
2,0.439880,3.892548,0.999945,0.945496,3.524902,0.999638,1.928741,0.726379,0.999153,0.970154,...,0.798141,1.392517,0.999419,2.681183,0.000000,0.986211,3.590607,2.148987,0.987800,11.4
3,7.703064,0.000000,0.999765,5.919373,0.000000,0.999534,4.751312,0.000000,0.999468,6.556610,...,1.855469,1.921738,0.999285,2.977997,0.041992,0.981083,3.765533,2.638062,0.994152,11.4
4,17.169312,0.000000,0.999855,20.999176,0.000000,0.999252,17.283508,0.000000,0.998945,13.850159,...,0.000000,4.927261,0.999156,0.000000,0.000000,0.983985,1.823181,0.067993,0.995609,11.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102495,1.339661,2.253326,0.999232,3.207214,5.388489,0.999924,3.782471,2.128571,0.999492,8.150452,...,0.582886,0.000000,0.999563,0.424469,0.375305,0.998793,4.226929,0.000000,0.998882,88.3
1102496,1.342590,4.823639,0.999680,1.945129,3.831635,0.999842,0.461548,1.584595,0.999562,3.117249,...,0.348389,0.000000,0.999529,0.385101,0.000000,0.998716,0.232422,-0.049561,0.998840,88.3
1102497,12.412659,0.000000,0.999622,9.379395,0.000000,0.999950,6.598083,2.645782,0.999656,7.575562,...,-0.155914,-1.260956,0.999215,0.496552,-0.301575,0.999035,0.308044,0.000000,0.998980,88.3
1102498,0.000000,0.000000,0.999570,0.000000,0.000000,0.999939,0.000000,0.000000,0.999616,0.000000,...,0.000000,-0.214752,0.999225,0.000000,-0.073730,0.998865,0.000000,0.000000,0.998603,88.3


In [25]:
filtered_data = data.filter(regex='x|y')
filtered_data

,"('nose', 'x')","('nose', 'y')","('H1R', 'x')","('H1R', 'y')","('H2R', 'x')","('H2R', 'y')","('H1L', 'x')","('H1L', 'y')","('H2L', 'x')","('H2L', 'y')",...,"('B2L', 'x')","('B2L', 'y')","('B3L', 'x')","('B3L', 'y')","('tail', 'x')","('tail', 'y')","('S2', 'x')","('S2', 'y')","('S1', 'x')","('S1', 'y')"
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.444275,2.021118,0.536194,1.981354,4.448181,4.355835,0.535675,1.918579,2.168060,3.959808,...,0.682312,0.721603,0.863861,0.081955,1.670120,0.212906,0.595001,0.076477,3.630341,2.596069
2,0.439880,3.892548,0.945496,3.524902,1.928741,0.726379,0.970154,0.231018,2.823761,3.261963,...,0.249634,1.100983,0.030807,0.525070,0.798141,1.392517,2.681183,0.000000,3.590607,2.148987
3,7.703064,0.000000,5.919373,0.000000,4.751312,0.000000,6.556610,0.000000,4.600220,0.000000,...,0.491577,2.587845,3.322083,5.370407,1.855469,1.921738,2.977997,0.041992,3.765533,2.638062
4,17.169312,0.000000,20.999176,0.000000,17.283508,0.000000,13.850159,0.000000,10.907776,0.000000,...,0.000000,1.244858,0.000000,4.321640,0.000000,4.927261,0.000000,0.000000,1.823181,0.067993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102495,1.339661,2.253326,3.207214,5.388489,3.782471,2.128571,8.150452,1.492737,6.739807,2.576172,...,0.000000,0.000000,0.000000,0.000000,0.582886,0.000000,0.424469,0.375305,4.226929,0.000000
1102496,1.342590,4.823639,1.945129,3.831635,0.461548,1.584595,3.117249,1.559113,3.390930,0.636566,...,0.203430,0.018829,0.000000,-0.047913,0.348389,0.000000,0.385101,0.000000,0.232422,-0.049561
1102497,12.412659,0.000000,9.379395,0.000000,6.598083,2.645782,7.575562,1.143738,9.298645,1.647430,...,-0.121002,-0.018829,-0.317688,-0.513275,-0.155914,-1.260956,0.496552,-0.301575,0.308044,0.000000
1102498,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-1.752960,-0.583069,-0.222778,0.000000,-0.214752,0.000000,-0.073730,0.000000,0.000000


## Base model

In [26]:
dropout_prob   = 0.5
embedding_size = 32
epoch_num      = 10
hidden_size    = 16
layer_num      = 2
learning_rate  = 1e-3
seq_size       = 25 #25 frames is equal to 1 second of video, maybe use 50?
pred_window    = 1

In [27]:
class SeqDataset(Dataset):
    def __init__(self, device, seq_size, dataframe, pred_window):
        super(SeqDataset, self).__init__()
        self.device      = device
        self.seq_size    = seq_size
        self.dataframe   = dataframe
        self.pred_window = pred_window
        self.target_data = dataframe.filter(regex='x|y')

    def __len__(self):
        return len(self.dataframe) - self.seq_size - 1

    def __getitem__(self, idx):
        in_seq = torch.tensor(self.dataframe.iloc[idx:idx + self.seq_size].values, dtype=torch.float, device=self.device)
        target_seq = torch.tensor(self.target_data.iloc[idx + self.pred_window:idx + self.seq_size + self.pred_window].values, dtype=torch.float, device=self.device)
        return in_seq, target_seq

In [28]:
# Split the data into train, validation, and test using the full video sizes (so that videos are not split into different sets)
train_data, test_data = train_test_split(filtered_data, test_size= 11250*int(0.2*98), shuffle=False)

# Assuming df is your dataframe with limb coordinate changes
scaler = StandardScaler()

# Fit the scaler on the training set
scaler.fit(train_data)

# Transform the training set
train_data_scaled = scaler.transform(train_data)
train_data = pd.DataFrame(train_data_scaled.reshape(train_data.shape), columns=train_data.columns)

# When you're ready to test the model, transform the test set
test_data_scaled = scaler.transform(test_data)
test_data = pd.DataFrame(test_data_scaled.reshape(test_data.shape), columns=test_data.columns)
train_data

,"('nose', 'x')","('nose', 'y')","('H1R', 'x')","('H1R', 'y')","('H2R', 'x')","('H2R', 'y')","('H1L', 'x')","('H1L', 'y')","('H2L', 'x')","('H2L', 'y')",...,"('B2L', 'x')","('B2L', 'y')","('B3L', 'x')","('B3L', 'y')","('tail', 'x')","('tail', 'y')","('S2', 'x')","('S2', 'y')","('S1', 'x')","('S1', 'y')"
0,-0.000052,0.000199,-0.000146,0.000217,-0.000170,0.000244,-0.000078,0.000233,-0.000089,0.000257,...,0.000040,0.000056,-0.000052,0.000033,-0.000031,-0.000021,-0.000087,0.000075,-0.000024,0.000202
1,0.030136,0.202595,0.042665,0.222129,0.392238,0.578631,0.048169,0.233582,0.221666,0.540242,...,0.043725,0.067129,0.044490,0.005059,0.093737,0.014304,0.033871,0.005106,0.435697,0.466434
2,0.029837,0.390002,0.075345,0.395007,0.169979,0.096696,0.087301,0.028331,0.288733,0.445079,...,0.016023,0.102393,0.001537,0.032235,0.044780,0.093670,0.152935,0.000075,0.430928,0.386141
3,0.523368,0.000199,0.472476,0.000217,0.418980,0.000244,0.590458,0.000233,0.470434,0.000257,...,0.031513,0.240598,0.171238,0.329391,0.104144,0.129277,0.169875,0.002837,0.451923,0.473975
4,1.166596,0.000199,1.676498,0.000217,1.524541,0.000244,1.247368,0.000233,1.115588,0.000257,...,0.000040,0.115766,-0.000052,0.265072,-0.000031,0.331494,-0.000087,0.000075,0.218798,0.012413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888745,-0.102628,0.000575,-0.036128,0.013897,-0.153636,0.000603,0.004996,0.025617,0.002674,0.020083,...,0.000040,-0.006168,-0.000052,-0.006652,-0.000031,-0.002033,-0.000087,-0.015013,-0.000024,0.061922
888746,-0.011756,1.254310,-0.906201,1.062190,-1.062746,0.695131,0.118954,1.505482,0.543744,0.715300,...,0.000040,-0.023209,-0.000052,-0.260603,-0.000031,-0.035382,-0.000087,-0.188087,-0.000024,0.013610
888747,-0.246973,1.353019,-0.400740,1.087945,-1.114228,1.109223,-0.000078,1.051559,-0.000089,0.623966,...,0.000040,-0.548576,-0.106311,-0.352955,-0.292687,-0.362257,-0.051875,-0.398471,-0.793082,0.000202
888748,-0.014951,0.000199,-0.034310,0.000217,-0.048630,0.000244,-0.000078,0.000233,-0.000089,0.000257,...,-0.036248,-0.015400,-0.011372,-0.008974,-0.009010,-0.005296,-0.013693,-0.013234,-0.084949,-0.013207


In [29]:

n_train_vids = len(train_data)/11250
train_data, val_data = train_test_split(train_data, test_size = 11250 * int(0.1 * n_train_vids), shuffle=False)

# Create SeqDataset instances for the train, validation, and test sets
train_dataset = SeqDataset(device, seq_size, train_data, pred_window)
val_dataset = SeqDataset(device, seq_size, val_data, pred_window)
test_dataset = SeqDataset(device, seq_size, test_data, pred_window)

# Create DataLoader instances for batching
batch_size = 64  # Adjust to your desired batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [30]:
class Model(nn.Module):
    def __init__(self, dropout_prob, hidden_size, layer_num, input_size, output_size):
        super(Model, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.gru       = nn.GRU(input_size, hidden_size, layer_num, batch_first=True, dropout=dropout_prob)
        self.linear    = nn.Linear(hidden_size, output_size)

    def forward(self, in_sequence, hidden_state=None):
#         embedding_seq            = self.embedding(in_sequence)
        hidden_seq, hidden_state = self.gru(in_sequence, hidden_state)
        out_seq                  = self.linear(hidden_seq)
        return out_seq, hidden_state
    
    def draw(self, in_sequence, logit_temp=1.0):
        out_seq, _  = self(in_sequence)
        prob_dist   = torch.softmax(out_seq[0, -1] / logit_temp, 0)
        rand_sample = torch.multinomial(prob_dist, 1).item()                   
        return rand_sample

In [31]:
input_size = next(iter(train_loader))[0].size(-1)
print("Input size:", input_size)
output_size = next(iter(train_loader))[1].size(-1)
print("Output size:", output_size)
model = Model(dropout_prob, hidden_size, layer_num, input_size, output_size).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

Input size: 28
Output size: 28


In [32]:
min_loss = float("inf")

for epoch in range(epoch_num):
    
    model.train()
    train_loss = 0.0
    
    i = 0
    for in_seq, target_seq in tqdm(train_loader, desc=f"Epoch {epoch}/{epoch_num}"):
        
        out_seq, _  = model(in_seq)
        loss        = criterion(out_seq, target_seq)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1

    train_loss       /= len(train_loader)
    # train_perplexity  = np.exp(train_loss)
    print(f"Train loss: {train_loss:.4f}")

    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for in_seq, target_seq in val_loader:
            out_seq, _  = model(in_seq)
            loss        = criterion(out_seq, target_seq)
            val_loss   += loss.item()

    val_loss       /= len(val_loader)
    # val_perplexity  = np.exp(val_loss)
    print(f"Val loss: {val_loss:.4f}")

    if val_loss < min_loss:
        min_loss = val_loss
        torch.save(model.state_dict(), "../Models/model_cc_norm_dropped_cols.pt")

Epoch 0/10: 100%|████████████████████████████████████████████████████████████████| 12655/12655 [06:04<00:00, 34.73it/s]


Train loss: 0.8354
Val loss: 1.3314


Epoch 1/10: 100%|████████████████████████████████████████████████████████████████| 12655/12655 [05:00<00:00, 42.11it/s]


Train loss: 0.8238
Val loss: 1.3115


Epoch 2/10: 100%|████████████████████████████████████████████████████████████████| 12655/12655 [04:51<00:00, 43.35it/s]


...

Train loss: 0.8160
Val loss: 1.3009


Epoch 8/10: 100%|████████████████████████████████████████████████████████████████| 12655/12655 [04:39<00:00, 45.36it/s]


Train loss: 0.8157
Val loss: 1.3005


Epoch 9/10: 100%|████████████████████████████████████████████████████████████████| 12655/12655 [04:50<00:00, 43.51it/s]


Train loss: 0.8156
Val loss: 1.3004


In [33]:
# model.load_state_dict(torch.load("../Models/model_cc_norm_dropped_cols.pt"))
model.eval

<bound method Module.eval of Model(
  (gru): GRU(28, 16, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=16, out_features=28, bias=True)
)>

In [34]:
test_loss = 0.0

for in_seq, target_seq in tqdm(test_loader):
    out_seq, _ = model(in_seq)
    loss = criterion(out_seq, target_seq)
    test_loss += loss.item()

test_loss /= len(test_loader)
print(f"Test loss: {test_loss:.4f}")

100%|██████████████████████████████████████████████████████████████████████████████| 3339/3339 [01:04<00:00, 51.53it/s]

Test loss: 0.6916


In [68]:
model.load_state_dict(torch.load("../Models/model_baseline.pt"))
model.eval

<bound method Module.eval of Model(
  (gru): GRU(43, 16, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=16, out_features=28, bias=True)
)>

In [69]:
test_loss = 0.0

for in_seq, target_seq in tqdm(test_loader):
    out_seq, _ = model(in_seq)
    loss = criterion(out_seq, target_seq)
    test_loss += loss.item()

test_loss /= len(test_loader)
print(f"Test loss: {test_loss:.4f}")

100%|██████████████████████████████████████████████████████████████████████████████| 3340/3340 [00:36<00:00, 91.03it/s]

Test loss: 952.3952


In [74]:
model.load_state_dict(torch.load("../Models/model_baseline_dropped_cols.pt"))
model.eval

<bound method Module.eval of Model(
  (gru): GRU(28, 16, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=16, out_features=28, bias=True)
)>

In [75]:
test_loss = 0.0

for in_seq, target_seq in tqdm(test_loader):
    out_seq, _ = model(in_seq)
    loss = criterion(out_seq, target_seq)
    test_loss += loss.item()

test_loss /= len(test_loader)
print(f"Test loss: {test_loss:.4f}")

100%|██████████████████████████████████████████████████████████████████████████████| 3340/3340 [00:40<00:00, 82.70it/s]

Test loss: 1070.2398


## Longer sequence

In [6]:
dropout_prob   = 0.5
embedding_size = 32
epoch_num      = 10
hidden_size    = 16
layer_num      = 2
learning_rate  = 1e-3
seq_size       = 50 #25 frames is equal to 1 second of video, maybe use 50?
pred_window    = 25
shift_size     = 25

In [7]:
class SeqDataset(Dataset):
    def __init__(self, device, dataframe, seq_size, pred_window, shift_size):
        super(SeqDataset, self).__init__()
        self.device = device
        self.seq_size = seq_size  # Input sequence length
        self.shift_size = 25  # Shift for the start of each new sequence
        self.pred_window = pred_window  # Additional frames for prediction
        self.dataframe = dataframe
        self.target_data = dataframe.filter(regex='x|y')

    def __len__(self):
        # The total number of sequences is adjusted for shifting the sequence start by self.shift_size
        return (len(self.dataframe) - self.seq_size - self.pred_window) // self.shift_size + 1

    def __getitem__(self, idx):
        # Calculate the actual starting index for the sequence based on the shift size
        start_idx = idx * self.shift_size
        end_idx = start_idx + self.seq_size
        target_end_idx = end_idx + self.pred_window

        # Ensure we don't exceed the bounds of the dataframe
        if target_end_idx > len(self.dataframe):
            target_end_idx = len(self.dataframe)
            end_idx = target_end_idx - self.pred_window

        in_seq = torch.tensor(self.dataframe.iloc[start_idx:end_idx].values,
                              dtype=torch.float, device=self.device)
        target_seq = torch.tensor(self.target_data.iloc[start_idx+self.pred_window:target_end_idx].values,
                                  dtype=torch.float, device=self.device)
        return in_seq, target_seq

In [ ]:
# Split the data into train, validation, and test using the full video sizes (so that videos are not split into different sets)
train_data, test_data = train_test_split(filtered_data, test_size= 11250*int(0.2*98), shuffle=False)

# Assuming df is your dataframe with limb coordinate changes
scaler = StandardScaler()

# Fit the scaler on the training set
scaler.fit(train_data)

# Transform the training set
train_data_scaled = scaler.transform(train_data)
train_data = pd.DataFrame(train_data_scaled.reshape(train_data.shape), columns=train_data.columns)

# When you're ready to test the model, transform the test set
test_data_scaled = scaler.transform(test_data)
test_data = pd.DataFrame(test_data_scaled.reshape(test_data.shape), columns=test_data.columns)
train_data

In [8]:
n_train_vids = len(train_data)/11250
train_data, val_data = train_test_split(train_data, test_size = 11250 * int(0.1 * n_train_vids), shuffle=False)

# Create SeqDataset instances for the train, validation, and test sets
train_dataset = SeqDataset(device, train_data, seq_size, pred_window, shift_size)
val_dataset = SeqDataset(device, val_data, seq_size, pred_window, shift_size)
test_dataset = SeqDataset(device, test_data, seq_size, pred_window, shift_size)

# Create DataLoader instances for batching
batch_size = 64  # Adjust to your desired batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [9]:
class Model(nn.Module):
    def __init__(self, dropout_prob, hidden_size, layer_num, input_size, output_size):
        super(Model, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.gru       = nn.GRU(input_size, hidden_size, layer_num, batch_first=True, dropout=dropout_prob)
        self.linear    = nn.Linear(hidden_size, output_size)

    def forward(self, in_sequence, hidden_state=None):
#         embedding_seq            = self.embedding(in_sequence)
        hidden_seq, hidden_state = self.gru(in_sequence, hidden_state)
        out_seq                  = self.linear(hidden_seq)
        return out_seq, hidden_state
    
    def draw(self, in_sequence, logit_temp=1.0):
        out_seq, _  = self(in_sequence)
        prob_dist   = torch.softmax(out_seq[0, -1] / logit_temp, 0)
        rand_sample = torch.multinomial(prob_dist, 1).item()                   
        return rand_sample

In [10]:
input_size = next(iter(train_loader))[0].size(-1)
print("Input size:", input_size)
output_size = next(iter(train_loader))[1].size(-1)
print("Output size:", output_size)
model = Model(dropout_prob, hidden_size, layer_num, input_size, output_size).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)
print(model)

Input size: 43
Output size: 28
Model(
  (gru): GRU(43, 16, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=16, out_features=28, bias=True)
)


In [11]:
min_loss = float("inf")

for epoch in range(epoch_num):
    
    model.train()
    train_loss = 0.0
    
    i = 0
    for in_seq, target_seq in tqdm(train_loader, desc=f"Epoch {epoch}/{epoch_num}"):
        
        out_seq, _  = model(in_seq)
        loss        = criterion(out_seq, target_seq)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1

    train_loss       /= len(train_loader)
    # train_perplexity  = np.exp(train_loss)
    print(f"Train loss: {train_loss:.4f}")

    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for in_seq, target_seq in val_loader:
            out_seq, _  = model(in_seq)
            loss        = criterion(out_seq, target_seq)
            val_loss   += loss.item()

    val_loss       /= len(val_loader)
    # val_perplexity  = np.exp(val_loss)
    print(f"Val loss: {val_loss:.4f}")

    if val_loss < min_loss:
        min_loss = val_loss
        torch.save(model.state_dict(), "../Models/model_cc_pred_25.pt")

Epoch 0/10: 100%|████████████████████████████████████████████████████████████████████| 506/506 [00:18<00:00, 27.34it/s]


Train loss: 139.9945
Val loss: 240.1483


Epoch 1/10: 100%|████████████████████████████████████████████████████████████████████| 506/506 [00:16<00:00, 30.27it/s]


Train loss: 139.8696
Val loss: 240.0997


Epoch 2/10: 100%|████████████████████████████████████████████████████████████████████| 506/506 [00:21<00:00, 23.06it/s]


...

Train loss: 139.7246
Val loss: 240.0083


Epoch 8/10: 100%|████████████████████████████████████████████████████████████████████| 506/506 [00:16<00:00, 30.56it/s]


Train loss: 139.6477
Val loss: 239.9857


Epoch 9/10: 100%|████████████████████████████████████████████████████████████████████| 506/506 [00:17<00:00, 28.45it/s]


Train loss: 139.6562
Val loss: 239.9908


In [38]:
model.load_state_dict(torch.load("../Models/model_pred_25.pt"))
model.eval

<bound method Module.eval of Model(
  (gru): GRU(43, 16, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=16, out_features=28, bias=True)
)>

In [39]:
test_loss = 0.0

for in_seq, target_seq in tqdm(test_loader):
    out_seq, _ = model(in_seq)
    loss = criterion(out_seq, target_seq)
    test_loss += loss.item()

test_loss /= len(test_loader)
print(f"Test loss: {test_loss:.4f}")

100%|███████████████████████████████████████████████████████████████████████████████| 133/133 [00:01<00:00, 102.39it/s]

Test loss: 11223.6390


In [12]:
# model.load_state_dict(torch.load("../Models/model_cc_pred_25.pt"))
model.eval

<bound method Module.eval of Model(
  (gru): GRU(43, 16, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=16, out_features=28, bias=True)
)>

In [13]:
test_loss = 0.0

for in_seq, target_seq in tqdm(test_loader):
    out_seq, _ = model(in_seq)
    loss = criterion(out_seq, target_seq)
    test_loss += loss.item()

test_loss /= len(test_loader)
print(f"Test loss: {test_loss:.4f}")

100%|████████████████████████████████████████████████████████████████████████████████| 133/133 [00:04<00:00, 30.12it/s]

Test loss: 120.8619
